In [ ]:
## 6中测试指标 https://blog.csdn.net/qq_35082030/article/details/105410478
## 开箱即用库 https://github.com/cfournie/segmentation.evaluation
## example
from segeval.window.pk import pk
from segeval.window.windowdiff import window_diff as WD
from segeval.similarity.boundary import boundary_similarity as B
from segeval.similarity.segmentation import segmentation_similarity as S

gold = [2, 3, 6]
h_list = [[5, 6], [2, 2, 7], [2, 3, 3, 3], [1, 1, 3, 1, 5]]
for n, h in enumerate(h_list):
    print("第%d次实验" % int(n + 1))
    print("1-Pk=%.3f, 1-WD=%.3f, B=%.3f, S=%.3f" % (pk(h, gold, one_minus=True),
            WD(h, gold, one_minus=True, window_size=2), B(h, gold), S(h, gold)))


In [1]:
import re
import pandas as pd 
from tqdm import tqdm
from segeval.window.pk import pk
from segeval.window.windowdiff import window_diff as WD
from segeval.similarity.boundary import boundary_similarity as B
from segeval.similarity.segmentation import segmentation_similarity as S

import sys
sys.path.append("..")
from utils.split_text import *

def init_spliter(method):
    if method == "cos_sim_spliter":
        from FlagEmbedding import FlagModel
        model = FlagModel('../resources/open_models/bge-large-zh-v1.5', query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：", use_fp16=True)
        spliter = BaseSpliter.use_subclass("cos_sim_spliter")(model)

    elif method == "doc_seg_model_spliter":
        from transformers import AutoModelForTokenClassification, AutoTokenizer
        model_name = '../resources/open_models/nlp_bert_document-segmentation_chinese-base'
        model = AutoModelForTokenClassification.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        spliter = BaseSpliter.use_subclass("doc_seg_model_spliter")(model, tokenizer)
    return spliter

df = pd.read_json("../resources/data/cleaned/CPTS.jsonl", lines=True)
df = df[df['split']=='test'].reset_index()

In [2]:
subject = "cos_sim_spliter"
spliter = init_spliter(subject)
results = []
for index, row in tqdm(df.iterrows()):
    text = "".join(row['sentences'])
    sentence_df = pd.DataFrame(row['sentences'], columns=['sentence'])
    sentence_df['start_idx'] = sentence_df['sentence'].apply(lambda x: re.search(re.escape(x), text).start())
    sentence_df['end_idx'] = sentence_df['sentence'].apply(lambda x: re.search(re.escape(x), text).end())
    chunk_df = spliter.cluster(sentence_df)
    pred, gt = chunk_df['sentence_count'].tolist(), row['masses']
    pk_score = pk(pred, gt, one_minus=True)
    try:
        wd_score = WD(pred, gt, one_minus=True, window_size=2)
    except Exception as e:
        print(f"WD calculation failed at index {index}: {e}")
        print(f"pred: {pred}, gt: {gt}")
        wd_score = None
    b_score = B(pred, gt)
    s_score = S(pred, gt)
    
    # 将指标保存到结果列表
    results.append({
        "index": index,
        "1-Pk": pk_score,
        "1-WD": wd_score,
        "B": b_score,
        "S": s_score
    })

# 转换结果列表为 DataFrame
results_df = pd.DataFrame(results)

# 计算各指标的平均值
results_df.loc["avg"] = {
    "index": "Average",
    "1-Pk": results_df["1-Pk"].mean(),
    "1-WD": results_df["1-WD"].mean(),
    "B": results_df["B"].mean(),
    "S": results_df["S"].mean()
}

/mnt/disk1/JXH/01_apps/miniforge3/envs/finbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
initial target device: 100%|██████████| 8/8 [00:28<00:00,  3.58s/it]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method 

pred: [5, 1, 10], gt: [3, 13]


Chunks: 100%|██████████| 8/8 [00:01<00:00,  7.74it/s]
2it [00:37, 15.79s/it]

pred: [3, 18, 2, 1], gt: [2, 9, 7, 6]


Chunks: 100%|██████████| 6/6 [00:00<00:00, 10.02it/s]
3it [00:38,  8.86s/it]

...

pred: [4], gt: [2, 1, 1]


Chunks: 100%|██████████| 8/8 [00:03<00:00,  2.53it/s]
1439it [31:35,  1.53s/it]

pred: [3, 2, 10], gt: [1, 4, 4, 4, 2]


Chunks: 100%|██████████| 7/7 [00:00<00:00,  7.28it/s]
1440it [31:36,  1.32s/it]

pred: [1, 7, 6], gt: [4, 5, 2, 3]


In [3]:
results_df.to_excel(f"results/文档主题分割-{subject}.xlsx")

In [4]:
results_df

,index,1-Pk,1-WD,B,S
0,0,0.6666666666666666666666666667,0.6428571428571428571428571429,0,0.8
1,1,0.5238095238095238095238095238,0.5909090909090909090909090909,0.1,0.8043478260869565217391304348
2,2,0.8888888888888888888888888889,0.8888888888888888888888888889,0.5,0.9
3,3,0.5185185185185185185185185185,0.6551724137931034482758620690,0.1428571428571428571428571429,0.8
4,4,0.7142857142857142857142857143,0.7142857142857142857142857143,0.5,0.9
...,...,...,...,...,...
1436,1436,0.8,0.8,0.75,0.9545454545454545454545454545
1437,1437,0.0,0.0,0,0.3333333333333333333333333333
1438,1438,0.4615384615384615384615384615,0.4615384615384615384615384615,0.2,0.7142857142857142857142857143
1439,1439,0.4166666666666666666666666667,0.4166666666666666666666666667,0.125,0.7307692307692307692307692308
